In [1]:
import os
import sys
sys.path.append('C:\\Users\\xpy\\work\\strategies2py\\AprilQuant')
import pandas as pd
from glob import glob
from collections import defaultdict
from concurrent import futures
from itertools import product
import pymongo

from signals import ChandelierSignalAdder
from backtest import run_bt_backtest, run_pd_backtest

In [2]:
def read_cache():
    root_path = '..\\..\\cache\\'
    categories = os.listdir(root_path)

    backtest_data = defaultdict(list)

    for category in categories:
        category_path = root_path + f'{category}\\'
        csv_files = os.listdir(category_path)
        for file in csv_files:
            df = pd.read_csv(category_path + file, parse_dates=['date'])
            backtest_data[category].append(df)
    
    return backtest_data

In [3]:
class Tester:
    backtest_data = read_cache()
    
    def __init__(self, params):
        # 数据标记
        self.params = params
        
    def test(self):
        category = self.params['data_label']['category']
        idx = self.params['data_label']['idx']
        
        df = self.backtest_data[category][idx]
        signal_adder = ChandelierSignalAdder(df)
        
        res = dict()
        res['params'] = self.params
        try:
            signal_adder.add_enter_signal(**self.params['enter_signal'])
        except ValueError as e:
            res['error'] = str(e)
        else:
            signal_adder.add_exit_signal(**self.params['exit_signal'])
            signal_adder.add_position_direction()

            res['bt_cum_ret'] = run_bt_backtest(signal_adder.df).reset_index().to_dict('list')
            res['pd_cum_ret'] = run_pd_backtest(signal_adder.df).reset_index().to_dict('list')
        
        return res

In [5]:
length_rng = range(40, 120, 10)
ema_length_rng = range(100, 180, 10)
trs_rng = (0.01*t for t in range(10, 20, 2))
lqk_width_rng = (0.1*w for w in range(1, 4, 1))
lqk_floor_rng = (0.1*f for f in range(3, 6, 1))

data_label_list = ({'category': category, 'idx': idx} 
                   for category, df_list in Tester.backtest_data.items()
                   for idx, _ in enumerate(df_list))
enter_signal_params_list = ({'length': length, 'ema_length': ema_length} 
                             for length, ema_length 
                            in product(length_rng, ema_length_rng))
exit_signal_params_list = ({'trs': trs, 'lqk_width': lqk_width, 'lqk_floor': lqk_floor} 
                           for trs, lqk_width, lqk_floor 
                           in product(trs_rng, lqk_width_rng, lqk_floor_rng))
params_lists = [{'data_label': data_label, 'enter_signal':enter_signal_params, 'exit_signal': exit_signal_params}
                for data_label, enter_signal_params, exit_signal_params 
                in product(data_label_list, enter_signal_params_list, exit_signal_params_list)]

In [6]:
sample_params = params_lists[0:4]
sample_testers = [Tester(params) for params in sample_params]

In [7]:
def run_test(tester):
    return tester.test()
    

res = list(map(run_test, sample_testers))

d:\python\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
d:\python\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
d:\python\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(in

In [8]:
r = 

In [11]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["AprilQuant"]
col = db['chandelier']
x = col.insert_one(res[1])

In [ ]:
pd.DataFrame(r['bt_cum_ret'])['index']

In [ ]:
list(res)